In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import data_table
data_table.enable_dataframe_formatter()

import cv2
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import torchvision.transforms as tf
import torchvision.models.segmentation 
import torch

#Referência: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.recall_score.html
from sklearn.metrics import recall_score
#Referência: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html
from sklearn.metrics import f1_score

**Definindo constantes**

In [ ]:
ImagesFolder="/content/drive/MyDrive/PROJETO-IA/DEV/corte/"

In [ ]:
height = 691
width = 1037

**Coletando todos os caminhos de imagens já utilizadas na parte do treino**

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/meu-projeto-nepcorte/dataset_dataframe.csv')
list_paths_used = list(df['Path'])

**Construindo uma lista de caminhos de imagens não utilizadas na parte de treino**

In [ ]:
list_paths_test = []
for i in range(1,8):
  folderPath = f'abate-{i}/'
  ListImages=os.listdir(os.path.join(ImagesFolder, folderPath))
  for path in ListImages:
    if (folderPath+path) not in list_paths_used and path != 'run.py' and path != 'delete-files.py':
      list_paths_test.append(folderPath+path)
print(list_paths_test)

['abate-1/A020', 'abate-1/A015', 'abate-1/A033(1)', 'abate-1/A023(2)', 'abate-1/a040', 'abate-1/A034', 'abate-1/A031', 'abate-1/A033 - OK', 'abate-1/A038', 'abate-1/A023', 'abate-1/A057', 'abate-1/A053', 'abate-1/A056(1)', 'abate-1/A044', 'abate-1/A058', 'abate-1/A054', 'abate-1/A051', 'abate-1/A063(1)', 'abate-1/A066', 'abate-1/A062 Pequeno', 'abate-1/A062(1)', 'abate-1/A065', 'abate-1/A059', 'abate-1/A072', 'abate-1/A061', 'abate-1/A094', 'abate-1/A086(1)', 'abate-1/A093', 'abate-1/A083', 'abate-1/A091', 'abate-1/A092', 'abate-1/A087', 'abate-1/A089', 'abate-1/A086', 'abate-1/A102', 'abate-1/A110', 'abate-1/A104', 'abate-1/A095', 'abate-1/A106', 'abate-1/A109', 'abate-1/A099(1)', 'abate-1/A099', 'abate-1/a115', 'abate-1/A116', 'abate-1/A122', 'abate-1/A119', 'abate-1/A118', 'abate-1/A120(1)', 'abate-1/A114', 'abate-1/A116(1)', 'abate-1/A120', 'abate-1/A127', 'abate-1/A130', 'abate-1/A128', 'abate-1/A124(2)', 'abate-1/A126', 'abate-1/a130(1)', 'abate-1/A46', 'abate-2/B023', 'abate-2/B

In [ ]:
print(len(list_paths_test))
list_paths_test = random.sample(list_paths_test,len(list_paths_test))
print(list_paths_test)

705
['abate-7/B084 (2)', 'abate-7/B018', 'abate-2/B032', 'abate-3/C061', 'abate-7/B052 (2)', 'abate-4/C115', 'abate-6/C016', 'abate-4/C097 (2)', 'abate-1/A038', 'abate-5/B164 (3)', 'abate-6/C099 (2)', 'abate-1/A092', 'abate-5/A182 (3)', 'abate-6/C033', 'abate-6/C028', 'abate-4/C112', 'abate-4/C117 (2)', 'abate-6/C062', 'abate-7/B014 (2)', 'abate-4/C108 (2)', 'abate-6/C017 (2)', 'abate-4/C093', 'abate-6/C022 (2)', 'abate-2/B044(1)', 'abate-1/A051', 'abate-6/C041', 'abate-4/C099 (3)', 'abate-5/B164', 'abate-5/B174 (3)', 'abate-4/C136 (2)', 'abate-6/C061', 'abate-7/B099 (1)', 'abate-5/B176', 'abate-6/C059', 'abate-5/A154', 'abate-7/B019 (2)', 'abate-4/C136 (3)', 'abate-4/C126 (2)', 'abate-1/A110', 'abate-5/A143 (2)', 'abate-1/A034', 'abate-5/B176 (2)', 'abate-6/C071 (3)', 'abate-1/A128', 'abate-7/B002 (1)', 'abate-4/C145 (2)', 'abate-6/C102 (2)', 'abate-4/C081', 'abate-1/A057', 'abate-7/B015 (2)', 'abate-7/B012', 'abate-5/B169 (2)', 'abate-5/A173', 'abate-6/C008 (2)', 'abate-5/A143', 'aba

**Lendo valores de desvio padrão e média de um arquivo**

In [ ]:
# Lendo a média e desvio padrão do arquivo "mean_and_std_list.txt"
mean_and_std_list_file = open("/content/drive/MyDrive/meu-projeto-nepcorte/mean_and_std.txt", "r")
content_list = mean_and_std_list_file.readlines()
mean = [float(integer) for integer in content_list[1].split(' ')[0:3]]
std = [float(integer) for integer in content_list[3].split(' ')[0:3]]
print(mean)
print(std)

[0.6600076038382257, 0.7012624583953048, 0.6471264305802913]
[0.18185697266551754, 0.19990861151114828, 0.15490782015523405]


**Função para criação de arquivo**

In [ ]:
def writeFile(l):
  with open(r'/content/drive/MyDrive/meu-projeto-nepcorte/images_metric_list.txt', 'w') as fp:
    for item in l:
      # write each item on a new line
      fp.write("%s\n" % str(item))
  print('File written!')

In [ ]:
modelPath = "/content/drive/MyDrive/meu-projeto-nepcorte/saved-weights/FCN_RESNET50/19.torch"  # Path to trained model

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
Net = torchvision.models.segmentation.fcn_resnet50(pretrained=True)  # Load net
Net.classifier[4] = torch.nn.Conv2d(512, 2, kernel_size=(1, 1), stride=(1, 1))  # Change final layer to 3 classes
Net = Net.to(device)  # Set net to GPU or CPU
Net.load_state_dict(torch.load(modelPath)) # Load trained model
Net.eval() # Set to evaluation mode

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=FCN_ResNet50_Weights.COCO_WITH_VOC_LABELS_V1`. You can also use `weights=FCN_ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/fcn_resnet50_coco-1167a1af.pth" to /root/.cache/torch/hub/checkpoints/fcn_resnet50_coco-1167a1af.pth


  0%|          | 0.00/135M [00:00<?, ?B/s]

FCN(
  (backbone): IntermediateLayerGetter(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequenti

In [ ]:
itr = 0
images_metrics = []
for folder in list_paths_test:
  imagePath = f"/content/drive/MyDrive/PROJETO-IA/DEV/corte/{folder}/img.png"

  Img = cv2.imread(imagePath) # load test image
  imgShape = Img.shape[0:2]
  height_orgin, width_orgin, d = Img.shape # Get image original size 

  transformImg=tf.Compose([tf.ToPILImage(),tf.Resize((height,width)),tf.ToTensor(),tf.Normalize((mean[0], mean[1], mean[2]), (std[0], std[1], std[2]))])

  Img = transformImg(Img)  # Transform to pytorch
  Img = torch.autograd.Variable(Img, requires_grad=False).to(device).unsqueeze(0)

  with torch.no_grad():
      Pred = Net(Img)['out']  # Run net
  Pred = tf.Resize((height_orgin, width_orgin))(Pred[0]) # Resize to origninal size
  seg = torch.argmax(Pred, 0).cpu().detach().numpy()  # Get  prediction classes

  maskPath = f"/content/drive/MyDrive/PROJETO-IA/DEV/corte/{folder}/label.png"
  Label = cv2.imread(maskPath,0) # load test image
  AnnMap = np.zeros(imgShape,np.float32)
  AnnMap[Label != 0] = 1 # set to 1 only the pixels that corresponds to the mask

  # Cálculo de recall
  recall = recall_score(AnnMap.flatten(), seg.flatten())

  # Cálculo de f1
  f1_macro = f1_score(AnnMap.flatten(), seg.flatten(), average='macro')
  f1_micro = f1_score(AnnMap.flatten(), seg.flatten(), average='micro')

  # Print
  print(f'Iteration: {itr} -> Recall: {recall}, F1 Macro: {f1_macro}, F1 Micro: {f1_micro}')

  images_metrics.append((folder,recall,f1_macro,f1_micro))
  # Salvar em arquivo
  if itr%10==0:
    writeFile(images_metrics)
  itr += 1

Iteration: 0 -> Recall: 0.9758380447066366, F1 Macro: 0.9878166544247495, F1 Micro: 0.9967666158514803
File written!
Iteration: 1 -> Recall: 0.9013837616185666, F1 Macro: 0.9627267743842427, F1 Micro: 0.9859503600823045
Iteration: 2 -> Recall: 0.9255897669155975, F1 Macro: 0.9778867998027732, F1 Micro: 0.9930866452510574
Iteration: 3 -> Recall: 0.9548078987614858, F1 Macro: 0.9810152018473258, F1 Micro: 0.9947077189071788
Iteration: 4 -> Recall: 0.9757423058263313, F1 Macro: 0.9859874917788265, F1 Micro: 0.9951383977051897
Iteration: 5 -> Recall: 0.9549089046943608, F1 Macro: 0.972631402021322, F1 Micro: 0.9932276372992398
Iteration: 6 -> Recall: 0.96759500487719, F1 Macro: 0.9874566684284081, F1 Micro: 0.9960811913258745
Iteration: 7 -> Recall: 0.9707401503255596, F1 Macro: 0.9890318812815853, F1 Micro: 0.9956648573245314
Iteration: 8 -> Recall: 0.8056760221072979, F1 Macro: 0.9424733479595395, F1 Micro: 0.996126235103738
Iteration: 9 -> Recall: 0.9069177924662261, F1 Macro: 0.9700112

2h e 17m para 293 iterações